<a href="https://colab.research.google.com/github/newsun87/google-colab-line_bot_learning/blob/main/linebot_liff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LIFF 註冊與管理
LIFF 就是一個可以在 LINE APP 內運作的 Web APP  平台

安裝 LINE Bot 相關套件

In [1]:
!pip install flask_ngrok pyngrok
!pip install Flask==2.0.3
!pip install line-bot-sdk
!ngrok authtoken 2Sh3hFOuE9JYeMNTsS6EzBxHPIL_6xdGTubFt7b9kSeUDFVYH

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=1a3ce810c32b312d05327b7c81bb512536a23df746a5172b65dcc5608b4f6d4a
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: Flask
    Found existing installation: Flask 2.2.5
    Uninstalling Flask-2.2.5:
      Successfully uninstalled Flask-2.2.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.7/736.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions


產生 ngrok url

In [ ]:
# 第一種寫法
from pyngrok import ngrok

# 開啟特定的 port
ngrok_tunnel = ngrok.connect(5000)  # 這裡的 5000 是你想要開啟的 port 號

# 取得公開的 URL
ngrok_url = ngrok_tunnel.public_url
print(f'Public URL: {ngrok_url}')

Public URL: https://afd1-34-125-199-30.ngrok-free.app


#### LIFF 申請
使用 LINE Developers 申請
1. 選擇 LINE Login
2. 點選 LIFF 新增

#### 範例 申請一個 LIFF APP 可以嵌入一個網頁
要求

直接在 LINE 用戶端輸入 LIFF URL， 直接點選後可以打開 PCHOME 網頁。用戶輸入 LIFF，會出現按鈕選單

解決方法：
直接申請一個 LIFF App，設定 PCHOME 網址並取的 LIFF APP URL

#### 範例 執行一個 LIFF Bot 可以嵌入一個 LIFF Web 網站
要求
當用戶輸入 『LIFF』，LINE APP 提供兩個按鈕選項 (Buttons) 的選單。
1. 第一個選項：顯示 PCHOME LIFF URL
2. 第二個選項：PCHOME (使用內嵌的方式， 導向 PCHOME 網頁)

In [2]:
from pyngrok import ngrok
from flask import Flask, request, abort
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import *
import json, os, time, requests

def get_ngrokurl():
   ngrok_tunnel = ngrok.connect(5000)  # 這裡的 5000 是你想要開啟的 port 號
   # 取得公開的 URL
   ngrok_url = ngrok_tunnel.public_url
   return ngrok_url

ngrok_url = get_ngrokurl()
print(ngrok_url)

# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'

# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url)

app = Flask(__name__)
# 監聽所有來自 LINE Bot Channel 的 Post Request
@app.route("/", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    print(f"訊息內容 {body}")
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    # 驗證訊息正確性
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.message.text == 'LIFF':
        buttons_template_message = TemplateSendMessage(
         alt_text = '我是一個按鈕模板',
         template = ButtonsTemplate(
            thumbnail_image_url = 'https://i.imgur.com/mMEP9oC.png',
            text = 'LIFF 測試',
            actions = [
                MessageAction(
                    label = '顯示PCHOME LIFF URL',
                    text = 'https://liff.line.me/1654118646-BEy9ne26'
                ),
                # 跳轉到URL
                URIAction(
                    label = 'PCHOME',
                    uri = 'https://liff.line.me/1654118646-BEy9ne26'  # 跳轉到的url
                )
            ]
         )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template_message )

if __name__ == "__main__":
  app.run()

https://e43c-34-148-217-66.ngrok-free.app


<ipython-input-2-ac539517dae5>:27: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-2-ac539517dae5>:29: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(channel_secret)
<ipython-input-2-ac539517dae5>:31: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.set_w

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {"destination":"Uef68021487469a005636e72e466f9acc","events":[{"type":"message","message":{"type":"text","id":"475908007151534292","quoteToken":"QoGDmshQhAP-UeIop5QF8UFIcmad3vXh1l8JtCU9apKns9NBBXeAuaFp-6idaLHi9WtbkI5H7OIofrRusNhwVw98w8vzr2bnohYyQhZNQctA45m0LUO5GGfXh5h8iDaK8xIrPpJmCy1R271-xcABlQ","text":"LIFF"},"webhookEventId":"01HBZF9XWXQ9WEDWVA0EREE6YV","deliveryContext":{"isRedelivery":false},"timestamp":1696494581166,"source":{"type":"user","userId":"Ubf2b9f4188d45848fb4697d41c962591"},"replyToken":"3346fef19bce497cac2bf6ab4e25e3b1","mode":"active"}]}


<ipython-input-2-ac539517dae5>:73: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, buttons_template_message )
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 08:29:42] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 08:30:12] "POST / HTTP/1.1" 200 -


訊息內容 {"destination":"Uef68021487469a005636e72e466f9acc","events":[{"type":"message","message":{"type":"text","id":"475908058170785812","quoteToken":"-1uhqoeDyXp2ALtWbAKeKPs-mkEcBwWLrOe_JjFzW8QgtIfZZi1CoxqjTRCdxWZUWhLX_vu4SGOV1ptXpIiVYPRaQt9QCp0AVFpuuHibc0xPwMno1kUSyf_DFzhMmk_MPeO2RgItYd4CwoYHLPHhxg","text":"https://liff.line.me/1654118646-BEy9ne26"},"webhookEventId":"01HBZFAVP68QJNGZW3DPZ5W9AC","deliveryContext":{"isRedelivery":false},"timestamp":1696494611659,"source":{"type":"user","userId":"Ubf2b9f4188d45848fb4697d41c962591"},"replyToken":"456e17c5c52a43108bdbd46033e62f67","mode":"active"}]}


#### 範例 設計一個 LIFF APP 可以嵌入自己設計的網頁
要求
在 LINE APP 輸入 HTML page 文字，LINE Bot 回應  LIFF APP URL ， 點選後顯示內嵌網頁

In [3]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort, render_template, url_for
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import *
import json, os, time, requests
from google.colab import drive
import os, time, json
import requests

# 掛載 Google Drive
drive.mount('/content/drive')

def get_ngrokurl():
    os.system("ps aux | grep ngrok.exe | awk '{print $2}' | xargs kill -9")
    os.system("nohup ngrok http 5000&")
    time.sleep(3)
    public_url = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    return public_url

ngrok_url = get_ngrokurl()
print(ngrok_url)

# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'

# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url + '/callback')

file_path = '/content/drive/My Drive/Colab Notebooks/\
LINE_Bot/templates'
app = Flask(__name__, template_folder=file_path)
@app.route('/') # 根路徑路由請求
def showIndex():
  # 回應網頁 index.html
  return render_template('index.html', name='')
# /say_hello 路由路徑請求 (接收網頁送來的表單資料)
@app.route('/say_hello', methods=['POST'])
def submit():
 name = request.form.get('username') # 取得表單中填寫的資料
 # 回應用戶的資料
 return render_template('index.html', name=name)

# 監聽所有來自 LINE Bot Channel 的 POST Request
@app.route("/callback", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    print(f"訊息內容 {body}")
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    # 驗證訊息正確性
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    global ngrok_url
    text = event.message.text
    if text == 'Hi':
      LIFF_URL = 'https://liff.line.me/1654118646-D0Z831aj'
      message = TextSendMessage(text = 'LIFF URL = ' + LIFF_URL)
      line_bot_api.reply_message(event.reply_token, message)

if __name__ == "__main__":
  app.run()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
https://ee66-34-148-217-66.ngrok-free.app


<ipython-input-3-9b9f2c26293b>:34: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-3-9b9f2c26293b>:36: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(channel_secret)
<ipython-input-3-9b9f2c26293b>:38: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.set_w

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {"destination":"Uef68021487469a005636e72e466f9acc","events":[{"type":"message","message":{"type":"text","id":"475913697949122595","quoteToken":"z5TEqxd-aeBqY2MNKdXsqfR1rc1orExXrxuK392FY2v20Ulvl0J4HBjJ9CFi_gjK9ZeZh0dH3Zh-tMXv0Jk4qYWZxDLJwk3nZDo_-apYXa1ErZKTOl-VI4rdlR-oj5_uYMecTXFHytGafu48tbLmOA","text":"Hi"},"webhookEventId":"01HBZJHEBBMCEWFVFM6K25WFXN","deliveryContext":{"isRedelivery":false},"timestamp":1696497973123,"source":{"type":"user","userId":"Ubf2b9f4188d45848fb4697d41c962591"},"replyToken":"a5316f05102f4f6aaf468a9c36e9a791","mode":"active"}]}


<ipython-input-3-9b9f2c26293b>:77: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, message)
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 09:26:14] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 09:27:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 09:27:17] "GET /liff-starter.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 09:27:17] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 09:27:18] "GET /liff-starter.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 09:27:18] "GET /favicon.ico HTTP/1.1" 404 -
